In [1]:
from thrift4DL.client import VisionClient
import numpy as np
from zaailabcorelib.ztools import encode_image, load_image
import requests
import json

### RESTFul API

In [10]:
def test_api(host, port, image_hex):
    url = f'http://{host}:{port}/v1/predict/'
    js = {"image": image_hex}
    response = requests.post(url, json=js)    
    return response.json()

In [11]:
img_arr = load_image('../example/mnist/num5.png')
img_hex = encode_image(img_arr)

In [12]:
response = test_api('0.0.0.0', '8100', img_hex)

In [13]:
response

{'content': {'pred_num': 5, 'pred_score': 0.9998825788497925},
 'error_code': 0,
 'error_message': 'Successful'}

### TCP

In [14]:
client = VisionClient(host='127.0.0.1', port='9090')

In [15]:
result = client.predict(img_hex)

In [16]:
result

'{"error_code": 0, "error_message": "Successful", "content": {"pred_num": 5, "pred_score": 0.9998825788497925}}'

In [17]:
client.ping()

In [18]:
%timeit client.predict(img_hex)

4.42 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
from ailabtools.ailab_multiprocessing import pool_worker

In [20]:
%timeit -n 1 -r 5 ret = pool_worker(target=client.predict, inputs=[img_hex]*3000, num_worker=20)

100%|██████████| 3000/3000 [00:04<00:00, 746.06it/s]


4.3 s ± 161 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [24]:
ret = pool_worker(target=client.predict, inputs=[img_hex]*1000, num_worker=20)

100%|██████████| 1000/1000 [00:01<00:00, 740.72it/s]


In [25]:
len(ret)

1000

In [26]:
0.4

0.4

In [27]:
result = client.predict(img_hex + "gg")

In [28]:
result

'{"error_code": -1, "error_message": "Internal Error", "content": ""}'